In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from statistics import mode, mean
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")
store_df = pd.read_csv("../input/store.csv")

In [ ]:
store_df.shape

In [ ]:
store_df.head()

In [ ]:
store_df.isna().sum()

In [ ]:
comp_month_lists = store_df[store_df['CompetitionOpenSinceMonth'].isnull()].index.tolist()


In [ ]:
for i in (comp_month_lists):
    store_type = store_df['StoreType'].iloc[i]
    my_list = list(store_df['CompetitionOpenSinceMonth'][store_df['StoreType'] == store_type])
    store_df['CompetitionOpenSinceMonth'].iloc[i] = max(set(my_list), key=my_list.count)

In [ ]:
store_df.isna().sum()

In [ ]:
store_df.iloc[11]

In [ ]:
store_df.apply(lambda x: len(x.unique()))

In [ ]:
store_df['StoreType'].value_counts()

In [ ]:
store_df['Assortment'].value_counts()

In [ ]:
store_df['CompetitionOpenSinceMonth'].value_counts()

In [ ]:
store_df['CompetitionOpenSinceYear'].value_counts()

In [ ]:
store_df['Promo2'].value_counts()

In [ ]:
store_df['Promo2SinceWeek'].value_counts()

In [ ]:
store_df['Promo2SinceYear'].value_counts()

In [ ]:
store_df['PromoInterval'].value_counts()

In [ ]:
train_df.head()

In [ ]:
train_df.shape

In [ ]:
train_df.isna().sum()

In [ ]:
train_df.apply(lambda x: len(x.unique()))

In [ ]:
train_df['DayOfWeek'].value_counts()

In [ ]:
train_df['Open'].value_counts()

In [ ]:
train_df['Promo'].value_counts()

In [ ]:
train_df['StateHoliday'].value_counts()

In [ ]:
train_df['SchoolHoliday'].value_counts()

In [ ]:
store_df['CompetitionDistance'].fillna(store_df['CompetitionDistance'].mean(), inplace=True)

In [ ]:
store_df.isna().sum()

In [ ]:
store_df = store_df[['Store','StoreType', 'Assortment', 'CompetitionDistance', 'Promo2', 'CompetitionOpenSinceMonth']]
store_df.head()

In [ ]:
train_df['StateHoliday'][train_df['StateHoliday']==0]="0"

In [ ]:
train_df['StateHoliday'].value_counts()

In [ ]:
train_df.head()

In [ ]:
train_df.tail()

In [ ]:
test_df.apply(lambda x: len(x.unique()))

In [ ]:
test_df['Open'].value_counts()

In [ ]:
test_df['StateHoliday'].value_counts()

In [ ]:
test_df['StateHoliday'][test_df['StateHoliday']==0]="0"

In [ ]:
test_df.isna().sum()

In [ ]:
test_df[test_df['Open'].isna()]

In [ ]:
open_lists = test_df[test_df['Open'].isnull()].index.tolist()
open_lists

In [ ]:
for i in (open_lists):
    d = test_df['DayOfWeek'].iloc[i]
    if(d !=6):
        test_df['Open'].iloc[i] = 1
    else:
        test_df['Open'].iloc[i] = 0

In [ ]:
test_df.isna().sum()

In [ ]:
test_df['Open'].value_counts()

In [ ]:
test_df.apply(lambda x: len(x.unique()))

In [ ]:
test_df['Open'].fillna(0, inplace = True)

In [ ]:
test_df['Open'] = test_df['Open'].astype(int)

In [ ]:
train_df['Open'] = train_df['Open'].astype(int)

In [ ]:
test_df['Open'].value_counts()

In [ ]:
train_closed_index = train_df[train_df['Open']==0].index

In [ ]:
test_closed_index = test_df[test_df['Open']==0].index

In [ ]:
test_df.tail()

In [ ]:
test_df.dtypes

In [ ]:
print(len(train_closed_index))
print(len(test_closed_index))

In [ ]:
test_zeros_ID = test_df['Id'][test_df['Open']==0].values

In [ ]:
test_zeros_ID[0:10]

In [ ]:
test_df.iloc[543]

In [ ]:
train_df_new = train_df.drop(train_closed_index)

In [ ]:
print("old train shape: ", train_df.shape)
print("new train shape: ",train_df_new.shape)

In [ ]:
len(train_closed_index)

In [ ]:
172817 + 844392

In [ ]:
test_new = test_df.drop(test_closed_index)

In [ ]:
test_new['Open'].value_counts()

In [ ]:
train_df_new.shape

In [ ]:
merged_train_df = pd.merge(train_df_new, store_df, on='Store', left_index=True)
merged_test_df = pd.merge(test_new, store_df, on="Store", left_index=True)

In [ ]:
merged_train_df.shape

In [ ]:
merged_train_df.head()

In [ ]:
merged_test_df.head()

In [ ]:
merged_test_df = merged_test_df.sort_values(by=['Id'])

In [ ]:
merged_test_df.shape

In [ ]:
merged_test_df.head()

In [ ]:
merged_test_df.tail()

In [ ]:
#train_df['Year'] = pd.DatetimeIndex(train_df['Date']).year
merged_train_df['Month'] = pd.DatetimeIndex(merged_train_df['Date']).month  
merged_train_df['Day'] = pd.DatetimeIndex(merged_train_df['Date']).day

#test_df['Year'] = pd.DatetimeIndex(test_df['Date']).year  
merged_test_df['Month'] = pd.DatetimeIndex(merged_test_df['Date']).month  
merged_test_df['Day'] = pd.DatetimeIndex(merged_test_df['Date']).day

In [ ]:
merged_train_df['CompetitionDistance'] = (merged_train_df['CompetitionDistance'] - (merged_train_df['CompetitionDistance'].min()))/ (merged_train_df['CompetitionDistance'].max() - merged_train_df['CompetitionDistance'].min())
merged_test_df['CompetitionDistance'] = (merged_test_df['CompetitionDistance'] - (merged_test_df['CompetitionDistance'].min()))/ (merged_test_df['CompetitionDistance'].max() - merged_test_df['CompetitionDistance'].min())

In [ ]:
merged_test_df.head()

In [ ]:
del merged_train_df['Date']
del merged_train_df['Store']
del merged_test_df['Date']
del merged_test_df['Store']
target1 = merged_train_df['Sales']
del merged_train_df['Sales']
del merged_train_df['Customers']

In [ ]:
merged_train_df['CompetitionOpenSinceMonth'].value_counts()

In [ ]:
merged_train_df = pd.get_dummies(merged_train_df, columns=['DayOfWeek', 'StateHoliday', 'Month', 'Day', 'StoreType', 'Assortment', 'CompetitionOpenSinceMonth'])
merged_test_df = pd.get_dummies(merged_test_df, columns=['DayOfWeek', 'StateHoliday', 'Month', 'Day', 'StoreType', 'Assortment', 'CompetitionOpenSinceMonth'])

In [ ]:
merged_train_df.head()

In [ ]:
merged_test_df.head()

In [ ]:
merged_test_df.tail()

In [ ]:
merged_test_df['Month_1']= 0
merged_test_df['Month_2']= 0
merged_test_df['Month_3']= 0
merged_test_df['Month_4']= 0
merged_test_df['Month_5']= 0
merged_test_df['Month_6']= 0
merged_test_df['Month_7'] = 0

merged_test_df['Month_10']= 0
merged_test_df['Month_11']= 0
merged_test_df['Month_12'] = 0

merged_test_df['StateHoliday_b']= 0
merged_test_df['StateHoliday_c']= 0

In [ ]:
IDs = merged_test_df['Id']
del merged_test_df['Id']

In [ ]:
print(merged_train_df.shape)
print(merged_test_df.shape)

In [ ]:
merged_test_df.head()

In [ ]:
del merged_test_df['Open']
del merged_train_df['Open']

In [ ]:
test_closed_index

In [ ]:
merged_train_df.head()

In [ ]:
merged_train_df.shape

In [ ]:
merged_test_df.shape

In [ ]:
target1.shape

In [ ]:
scalarY_train = MinMaxScaler()
scalarX_train = MinMaxScaler()

scalarY_train.fit(np.array(target1).reshape(844392,1))
scalarX_train.fit(np.array(merged_train_df))
x_train =  scalarX_train.transform(np.array(merged_train_df))
y_train = scalarY_train.transform(np.array(target1).reshape(844392,1))

In [ ]:
x_train.shape   

In [ ]:
model = Sequential()
model.add(Dense(150,input_dim=77, activation='relu'))
model.add(Dense(150,activation='relu'))
model.add(Dense(77,activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='mse', optimizer='adam')
model.fit(x_train, y_train, epochs=10,verbose=0)

In [ ]:
test_closed_index

In [ ]:
merged_test_df.head()

In [ ]:
y_pred = model.predict(np.array(merged_test_df))  

In [ ]:
real_prediction = scalarY_train.inverse_transform(y_pred)

In [ ]:
submission_df = pd.DataFrame()

In [ ]:
submission_df['Id'] = merged_test_df.index+1

In [ ]:
submission_df['Sales'] = real_prediction

In [ ]:
submission_df.head()

In [ ]:
submission_df.tail(50)

In [ ]:
dummy =pd.DataFrame()
dummy['Id'] = test_closed_index +1
dummy['Sales'] = 0

In [ ]:
result = pd.concat([submission_df, dummy])

In [ ]:
result.head()

In [ ]:
result.tail()

In [ ]:
result = result.sort_values(by=['Id'])

In [ ]:
result.index = range(0,len(test_df))

In [ ]:
result

In [ ]:
result['Sales'] = result['Sales'].astype(int)

In [ ]:
result.to_csv('submission.csv',index=False)